# Intro
Install the required packages using uv:
```zsh
uv add numpy polars scipy
```

In [144]:
import numpy as np
import polars as pl
import scipy

Load the Algerian exports dataset from the text file

In [145]:
df = pl.read_csv("data/exports.txt", separator='\t', has_header=True)
df

Year,Exports
i64,f64
1960,39.043173
1961,46.244557
1962,19.793873
1963,24.684682
1964,25.084059
…,…
2013,33.209898
2014,30.219117
2015,23.171778


Generate naive forecasts - prediction for time step `t` is the observation from `t-1`

In [146]:
df = df.with_columns(pl.col('Exports').shift(1).alias('Naive'))
df

Year,Exports,Naive
i64,f64,f64
1960,39.043173,null
1961,46.244557,39.043173
1962,19.793873,46.244557
1963,24.684682,19.793873
1964,25.084059,24.684682
…,…,…
2013,33.209898,36.890548
2014,30.219117,33.209898
2015,23.171778,30.219117


Compute the mean squared error (mse) of naive forecasts

In [147]:
np.mean(np.square(np.array(df['Exports'])[1:] - np.array(df['Naive'])[1:]))

np.float64(35.7310080231692)

Write a function for simple ETS

In [ ]:
def ets(alpha, obs):
    '''
    alpha:  parameter of the ETS model
    obs:    'vector' of observations, which indices correspond to consecutive timesteps
    '''
    
    forecast = np.full(len(obs), np.nan) 
    forecast[1] = obs[0] # start with naive forecast

    for t in range(1, len(obs)-1):
        forecast[t+1] = alpha*obs[t] + (1-alpha)*forecast[t]
    
    return forecast

Now, find the optimal alpha (minimizing mse) with a simple grid search method. First, write a function that for a given alpha outputs mse of a simple ETS model on our exports data

In [149]:
def ets_exports_mse(alpha):
    obs = np.array(df['Exports'])
    forecast = ets(alpha, obs)
    return np.mean(np.square(forecast[1:] - obs[1:]))

Specify the grid of alpha values and prepare an object for storing the corresponding mse value

In [150]:
alpha_values = np.linspace(0.0, 1.0, 101)
mse_values = np.full(len(alpha_values), np.nan)

Iterate through the grid and calculate the mse for each alpha value

In [151]:
for i in range(0, len(alpha_values)):
    mse_values[i] = ets_exports_mse(alpha_values[i])

Print the minimal mse and the corresponding optimal alpha

In [152]:
print("Minimal mse: ", np.min(mse_values))
print("Optimal alpha: ", alpha_values[np.argmin(mse_values)])

Minimal mse:  35.00941685268732
Optimal alpha:  0.84


Now, optimize alpha uscing `scipy.optimize.minimize()` function

In [ ]:
opt_params = scipy.optimize.minimize(ets_exports_mse, [0.5], bounds = [(0, 1)])
print("Optimal parameters: ", opt_params.x)

Optimal parameters:  [0.83952679]


/var/folders/xy/dhxtkwz95bsf570h1dm0xmv00000gn/T/ipykernel_3584/2175954633.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  forecast[t+1] = alpha*obs[t] + (1-alpha)*forecast[t]


In [ ]:
def wrap_ets(params):
    ets_exports_mse(params[0])

Explain why we encoutered a warning in the line above and how the lambda function in the line below solves this issue

In [ ]:
opt_params = scipy.optimize.minimize(wrap_ets, [0.5], bounds = [(0, 1)])
print("Optimal parameters: ", opt_params.x)

Optimal parameters:  [0.83952679]


**Homework** 

Modify the `ets()` function so that instead of `alpha` it takes two paramters: `alpha` and `level0`, where `level0` will correspond to the first forecast (instead of `forecast[1] = obs[0]`)